In [1]:
def indexing(file): #tager et filobjekt som argument
    textlist = file.read().lower().split() #lægger filens indhold ind på en liste i lowercase
    import string
    with open('stopord.txt') as stopord, open('forkortelser.txt') as forkortelser:
        forkortelser_list = forkortelser.read().split()
        stopord_list = stopord.read().split()
        
        for index, word in enumerate(textlist):
            if word not in forkortelser_list:
                textlist[index] = word.strip(string.punctuation) #fjerner tegn fra ord - ikke forkortelser
        textlist = [word for word in textlist if word not in stopord_list] #fjerner stopord fra tekstlisten
                
    return textlist

In [2]:
def file_wordfreq(textfiles): #tager en liste over tekstfiler som argument
    file_wordfreq = {} #tom dict til at komme filnavne og indlejret dict med ord og frekvenser i 
    for textfile in textfiles: #loop over hver tekstfil
        if 'stopord' in textfile or 'forkortelser' in textfile or 'forespoergsler' in textfile:
            continue #disse filer er ikke til at søge i, så funktionen springer dem over
        else:
            with open(textfile, encoding='utf-8-sig') as f: #åbner filen og fjerner BOM
                wordfreq = {} #tom dict til at komme ord og frekvenser i
                textlist = indexing(f) #ordene på listen indekseres med indexing-funktionen
                for word in textlist:
                    if word not in wordfreq:
                        wordfreq[word] = 1 #ord, der ikke allerede er i dicten, lægges ind med frekvens 1 
                    else:
                        wordfreq[word] += 1 #ord, der er i dicten, får lagt 1 til deres frekvens
                        
        file_wordfreq[textfile] = wordfreq 
        #hvert filnavn lægges ind som key i dicten file_wordfreq,
        #mens value er en indlejret dict med filens ord og deres frekvens

    return file_wordfreq

In [3]:
def search_routine():
    
    import os 
    textfiles = [el for el in os.listdir(os.getcwd()) if el.endswith('.txt')] 
    #listen over filer i cwd opdateres til kun at indeholde .txt-filer
    
    import pandas as pd
    df = pd.DataFrame(file_wordfreq(textfiles)) #dataframe over file_wordfreq
    df = df.fillna(0) #felter i df uden værdi gives 0 i værdi

    search_word = input('Hvad er dit søgeord? Skriv \'exit\' for at stoppe programmet: ').lower()
    if search_word == 'exit':
        return print('Du har lukket programmet.')
        
    try: #try gør, at programmet kan lave en exception i stedet for at crashe, hvis søgeordet ikke findes    
        search_results = {} #tom dict til søgeresultater
        for text in df.columns:
            if df.loc[search_word, text] > 0: #tjekker, om søgeordet forekommer ud fra teksten i dataframen
                search_results[text] = df.loc[search_word, text] 
                #filnavnet kommer ind som key og ordets frekvens kommer ind som value i dicten search_results           
        search_results_list = sorted(search_results.items(), key = lambda x: x[-1], reverse = True) 
        #listen er sorteret, så den går fra højest frekvens mod lavest
        #den består af tuples med filnavn først og ordets frekvens sidst
        
        print('\nDit søgeord', search_word.upper(), 'findes i følgende dokumenter:') 
        for index, el in enumerate(search_results_list):
            print(index + 1, el[0], 'med frekvensen', int(el[-1])) 
            #søgeordets frekvens i hver tekst printes og hver fil gives et tal, så de kan åbnes af brugeren
       
        search_files_list = input('''\nSkriv tallet ud for filnavnet for at se filindholdet.
        \nSkriv evt. flere tal adskilt af mellemrum eller skriv \'exit\' for at søge igen: ''').lower().split() 
        #brugerens filtal laves til en liste 
        if search_files_list == ['exit']:
            print('\nDu kan nu søge igen.')
        else:
            print()
            for num in search_files_list: #loop over brugerens angivne filtal
                with open(search_results_list[int(num)-1][0]) as f: 
                    #åbner den angivne fil ved at trække 1 fra brugerens tal, så det svarer til indekset
                    #første del af hver tuple på listen indeholder filnavnet
                    print(num, search_results_list[int(num)-1][0].upper()) #filtallet og filnavnet i caps
                    print(f.read())
                    print()
            print('\nDu kan nu søge igen.')
            
    except: #hvis søgeordet ikke findes, undgår programmet at crashe pga. denne exception
        with open('stopord.txt') as stopord:
            stopord_list = stopord.read().split()
            if search_word in stopord_list:
                print('Søgeordet', search_word.upper(), 'er et stopord og kan ikke søges på.\n')
            else:
                print('Søgeordet', search_word.upper(), 'forekommer ikke i nogen dokumenter.\n')
    
   
    
    search_routine()
    

In [9]:
search_routine()

Hvad er dit søgeord? Skriv 'exit' for at stoppe programmet: mad

Dit søgeord MAD findes i følgende dokumenter:
1 mad.txt med frekvensen 5
2 hverdag_med_diabetes.txt med frekvensen 3
3 mundhulen.txt med frekvensen 2
4 livsstil.txt med frekvensen 1
5 diabetes2.txt med frekvensen 1

Skriv tallet ud for filnavnet for at se filindholdet.
        
Skriv evt. flere tal adskilt af mellemrum eller skriv 'exit' for at søge igen: exit

Du kan nu søge igen.
Hvad er dit søgeord? Skriv 'exit' for at stoppe programmet: kost

Dit søgeord KOST findes i følgende dokumenter:
1 behandling_for_diabetes2.txt med frekvensen 3
2 mundhulen.txt med frekvensen 2
3 risiko.txt med frekvensen 1
4 hurtige_om_diabetes.txt med frekvensen 1

Skriv tallet ud for filnavnet for at se filindholdet.
        
Skriv evt. flere tal adskilt af mellemrum eller skriv 'exit' for at søge igen: exit

Du kan nu søge igen.
Hvad er dit søgeord? Skriv 'exit' for at stoppe programmet: næring
Søgeordet NÆRING forekommer ikke i nogen dokum